In [2]:
import geopandas as gpd
import folium 
import pandas as pd
import random 
from branca.element import Element, Figure

# creo due array vuoti
nomi_specie = []
tot_alberi = []

# leggo il dataframe
df = gpd.read_file('C:/Users/ronca/Desktop/pw_villaburi/mappa_interattiva/df_utente.geojson')

# creo un oggetto folium richiamando coordinate della villa, livelli di zoom e stile della mappa
map = folium.Map(location= [45.41908567793641, 11.042160973520623],
               zoom_start = 17,
               max_zoom = 19,
               min_zoom = 17,
               tiles = 'CartoDB Positron'
            )

#creo un HTML con un popup up interattivo, inserendo descrizione, foto e audio 
html = """
    <h4>VILLA BURI</h4>
    <p>
    Il parco di Villa Buri si estende per una superficie di 13 ettari ricca di flora e fauna. Infatti, il parco comprende, al suo interno, ben 1800 diverse piante, tra cui alcune piante secolari.
    Per lo svago e l’intrattenimento delle famiglie, nel parco si possono trovare campi da calcio e da pallavolo. Inoltre, il parco offre anche un percorso sensoriale nel bosco e comprende un orto botanico, 
    viene attualmente curato dall’Associazione Amici di Villa Bosco Buri.
    </p>
    <img src="https://upload.wikimedia.org/wikipedia/commons/5/5e/Verona_-_Villa_Buri.JPG"style="max-width:300px;">
"""
audio_file = 'https://github.com/illRoncc/illRoncc.github.io/raw/main/audio_villa_buri.mp3'

audio = Element(
    '<audio controls>'
    f'<source src="{audio_file}" type="audio/mpeg">'
    '</audio>'
)

popup = folium.Popup(html + audio.render(), max_width=300)
folium.Marker(location=[45.4197933, 11.0411905], 
              popup=popup, 
              icon=folium.Icon(icon='home', prefix='fa', color='red')).add_to(map)


# inserisco in una variabile la colonna di riferimento del dataframe
x = df['Nome comune'].value_counts()

# estraggo dall'array nomi_specie formato da coppia chiavi valori solo la chiave
for key, value in x.items():
    nomi_specie.append(key)

# confronto l'array tot_alberi con il dataframe in base alla colonna 'nome scientifico', cosi
# da creare un array dove sono presenti il numero di alberi dentro l'array nomi_specie
for i in range(len(nomi_specie)):
    tot_alberi.append(df.loc[df['Nome comune'] == nomi_specie[i]])

# setto colori random per layerare la mappa 
get_color = lambda n: ['#%06x' % random.randint(0, 0xFFFFFF) for _ in range(n)]
colors = get_color(66)

# setto i colori e i marker per le varie specie di alberi
i = 1
for specie in tot_alberi:
    if i-1 < len(colors):
        specie.explore(
            m = map, 
            color = colors[i-1],
            name = nomi_specie [i-1],
            style_kwds = {'style_function': lambda x: {'radius': pd.to_numeric(x['properties']['Diametro (cm)'])/15}}
        )
    else:
        pass
    i = i + 1

folium.LayerControl().add_to(map)

# salvo l'oggetto mappa in un file HTML
map.save('mappa.html')















